In [1]:
import spacy
import re
import pandas as pd
from string import punctuation

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import pipeline
from sklearn.metrics import confusion_matrix,classification_report

In [3]:
nlp = spacy.load("en_core_web_sm")

### Data Cleaning

In [4]:
# Strip
# Remove New line character
# Apply lower to text
# Tokenize
# Remove Stop words
# Remove punctuations (Use spacy is_punct. in NLTK , we can use regex subsitute re.sub('[^A-Za-z]',' ',review) before tokenization or use string.punctuation)
# Get Lemma

In [5]:
def clean_review(review):
    review=review.strip()
    review=re.sub('\n',' ',review)
    review=review.lower() # lower case
    review=nlp(review)
    review=[token for token in review if token.is_stop==False]  # remove stop words
    review=[token for token in review if token.is_punct==False]
    review=[token.lemma_ if token.lemma_ != "-PRON-" else token for token in review]
    return review
    

In [6]:
df=pd.read_csv('Restaurant_Reviews.tsv',delimiter='\t')

In [7]:
X,y=df['Review'],df['Liked']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,)

In [8]:
tfidf=TfidfVectorizer(tokenizer=clean_review)

In [9]:
svc=LinearSVC()

In [10]:
clf=pipeline.make_pipeline(tfidf,svc)

In [11]:
clf.fit(X_train,y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function clean_review at 0x000001A2DD99E1F8>)),
                ('linearsvc', LinearSVC())])

In [12]:
clf.score(X_test,y_test)

0.8

In [13]:
pd.DataFrame(confusion_matrix(y_test,clf.predict(X_test)),index=['Actual-0','Actual-1'],columns=['Prediction-0','Prediction-1'])

,Prediction-0,Prediction-1
Actual-0,94,23
Actual-1,27,106


In [14]:
print(classification_report(y_test,clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.78      0.80      0.79       117
           1       0.82      0.80      0.81       133

    accuracy                           0.80       250
   macro avg       0.80      0.80      0.80       250
weighted avg       0.80      0.80      0.80       250



## Test your own review

In [15]:
sample="""
Fit was slightly small, strap was abrasive feeling. Sandals looked good online but looked and felt cheap in person. I’m going back to Olukai...
"""

In [16]:
print('Cleaned Text:\n',clean_review(sample))
print('\n')
print('Review was ','POSITIVE' if clf.predict([sample])==[1] else 'NEGATIVE')

Cleaned Text:
 ['fit', 'slightly', 'small', 'strap', 'abrasive', 'feeling', 'sandal', 'look', 'good', 'online', 'look', 'feel', 'cheap', 'person', 'go', 'olukai']


Review was  NEGATIVE
